In [1]:
ADDR = 'Portfolio-11-20-2021.csv'

In [2]:
import pandas as pd

In [3]:
# lendo o arquivo, sem filtros

def read_file(addr):
    p = pd.read_csv(addr)
    p = p[p.Portfolio == "Bovespa"]
    p = p.reset_index()

    d = {'Id':'id', 'Symbol':'symbol', 'Type': 'type', 'Name':'name', 'Exchange':'exchange', 'Quantity':'qnt', 'Cost Per Share':'value', 'Date':'date', 'TimeOfDay': 'time', 'Notes        ':'notes'}
    p = p.rename(columns=d)

    p = p[['id', 'symbol', 'name', 'exchange', 'qnt', 'value', 'type', 'date', 'time', 'notes']]
    
    # convertion to 'date' format
    p.replace('\s*GMT[+-].*', '', regex=True, inplace=True)
    p.date = pd.to_datetime(p.date)
    
    return p


In [6]:
# imprimir operacoes acao e saldo + preco medio no final
ACAO = 'ABEV'
p = read_file(ADDR)

def check_exist(buf, pi):
    for bi in buf:
        if (bi.qnt == pi.qnt) and (bi.type == pi.type) and (bi.value == pi.value) and (abs((bi.date - pi.date).days) <= 1):
            return True
    return False


def get_acao(p, ACAO):
    # filtrando acoes ACAO
    p_ = p[[ACAO.upper() in s for s in p.symbol]]

    # listando por ordem de transacao (todas, antes de filtrar)

    saldo = 0
    valor = 0.
    
    buf   = []

    for pi_ in p_.iterrows():
        pi = pi_[1]

        if pd.isna(pi.qnt):
            continue

        if check_exist(buf, pi):
            continue

        buf.append(pi)
        # print(f"{pi.symbol}, {pi.date}, {pi.time}, {pi.type}, {pi.qnt}, {pi.value}, {pi.notes}")

        # check if is in buffer

        if pi.type == 'Buy':
            saldo += pi.qnt
            valor += pi.value*pi.qnt

        if pi.type == 'Sell':
            saldo -= pi.qnt
            valor -= pi.value*pi.qnt

            

    p_out = pd.DataFrame(buf)
    
    pm = 0.
    if abs(saldo) > 0.1:
        pm = valor/saldo
        
    print(pi.symbol, saldo, f'{valor:.2f}')
        
    return p_out, saldo
    
get_acao(p, 'embr');

EMBR3.SA 2000.0 10869.00


In [7]:
for s in p.symbol.unique():
    get_acao(p, s)

BIDI4.SA 100.0 -28278.00
ITSA4.SA 2500.0 26156.00
PETR4.SA 700.0 563.00
ABEV3.SA 0.0 -8330.00
CIEL3.SA 2500.0 11690.00
GOLL4.SA 0.0 -1953.00
AZUL4.SA 400.0 5911.00
EMBR3.SA 2000.0 10869.00
BBDC4.SA 0.0 -122.00
TRAD3.SA 11000.0 70679.00
BIDI11.SA 400.0 19300.00
MGLU3.SA 1100.0 10373.00
